# Clustering chemicals based on their descriptors

The scripts bellow use K-Means clustering the classify the merged FEMA-JECFA chemicals based on the merged descriptors obtained in [fema_jecfa_merge](fema_jecfa_merge.ipynb)

In [1]:
import os.path as path
import pickle

# Load FEMA database
BASE_DATA_PATH = path.join(path.expanduser('~'),
                           'Dropbox',
                           'bymt',
                           'data_dumps',
                           'chem_project')

merged_chemicals_path = path.join(BASE_DATA_PATH, 'fema_jecfa_merge', 'merged_chemicals.pkl')
with open(merged_chemicals_path, 'rb') as f:
    merged_chemicals = pickle.load(f)

Add unique id numbers to the chemicals in merged list so that I can relate their classification back to their chemical

In [2]:
from copy import deepcopy

def add_id(dicto_list):
    """
    Add unique id numbers to the dictos in dicto_list.
    Each id is the index of that dicto in the list.
    """
    new_list = deepcopy(dicto_list)
    for i, dicto in enumerate(new_list):
        dicto['id'] = i
    return new_list

In [3]:
id_chemicals = add_id(merged_chemicals)
print(id_chemicals[0]['id'])

0


Create list of tuples pairing chemical id with its merged descriptors as input for K-means clustering

In [14]:
def id_descriptor_lists(dicto_list):
    """
    Creates paired lists. One with ids, the other with merged descriptors
    """
    ids, descriptors = [], []
    for dicto in dicto_list:
        ids.append(dicto['id'])
        descriptors.append(dicto['merged descriptors'])
    return ids, descriptors

In [15]:
ids, descriptors = id_descriptor_lists(id_chemicals)
print('lengths make sense: {}' .format(len(descriptors)==len(id_chemicals)))

lengths make sense: True


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.75, 
                             stop_words='english',
                             norm='l2',
                             ngram_range=(1,2))

descriptor_vector = vectorizer.fit_transform(descriptors)
print(descriptor_vector.shape)

(2193, 4543)


In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=n, init='k-means++',
            max_iter=100, n_init=1, verbose=verbose)